# Cloud Networking
In today's data centers, networks are composed of more devices than ever before.
Servers, switches, routers, storage systems, and security appliances now exist
as virtual machines and virtual network appliances. These devices place a large
strain on traditional network management systems, as they are unable to provide
a scalable, automated approach to managing next-generation networks. Users now
expect more control and flexibility of the infrastructure with quicker provisioning,
all of which Cloud Networking promises to delive
## Functions Of Cloud Networking
Cloud Networking includes many technologies one would find in a data center,
including (**switching, routing, load balancing, firewalling, and virtual private networks**) aaS.
These features can be configured to leverage open source or commercial software,
and provide a cloud operator with all the tools necessary to build a functional and
self-contained cloud.Cloud Networking also provides a framework for
third-party vendors to build on and enhance the capabilities of the cloud


## Network Topologies Using Neutron
 Network Topologies using Neutron is an architecture which describes the network  which includes the private and public networks hsving their interefaces which are connected each other with L3 and L2 devices.



## Building our First Network Topology
Our initial network topology consists on creating two networks, each one connected to one instance.



![Image](https://github.com/ml6973/Cloud-Tutorial/blob/master/OpenStack/figures/case1.png?raw=True)



In [ ]:
%%bash
. /home/stack/devstack/openrc admin admin

neutron net-create my-private-network-1
neutron net-create my-private-network-2
neutron subnet-create my-private-network-1 --name my-private-subnet-1 --gateway 192.177.0.1 192.177.0.0/24
neutron subnet-create my-private-network-2 --name my-private-subnet-2 --gateway 192.178.0.1 192.178.0.0/24
nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone nova --nic net-name=my-private-network-1 my-instance-1
nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone nova --nic net-name=my-private-network-2 my-instance-2

PRIVATE_NET1_ID=$(neutron net-list | grep '| my-private-network-1' | awk '{print $2}')

PRIVATE_NET1_NS='qdhcp-'$PRIVATE_NET1_ID

NOVA_1=$(nova list | grep '| my-private-network-1' | awk '{print $12}')

echo $NOVA_1

sudo ip netns exec $PRIVATE_NET1_NS sshpass -p "cubswin:)" ssh  cirros@$NOVA_1


## Testing Network Connectivity

To perform network connectivity between instances, we will access **my_private_network_1** and request a ping to be performed from **my_instance_1** to **my_instance_2**. To perfom this action, run the following commands on the terminal:


In [ ]:
from IPython.display import IFrame
IFrame ("http://129.114.108.71:8888/terminals/1", width = 800, height=500)

## Output
In the given topology each instance is  connected to a private network but as it can be observed, there is no connectivity between instances.

## First Network Topology Clean-up

The code shown below will remove the previously created topology.

In [ ]:
%%bash
. /home/stack/devstack/openrc admin admin

nova delete my-instance-1
nova delete my-instance-2

#Private 1,2
PRIVATE_SUBNET_ID=$(neutron subnet-list | grep '| my-private-subnet1' | awk '{print $2}')
for i in `neutron port-list | grep -v '+' | grep -v 'fixed_ips' | awk '{print $2}'`;
do
  if [ "$i" != "|" ]
  then
    neutron port-update --device-owner None $i;
    neutron port-delete $i;
  fi
done;

neutron net-delete my-private-network-1
neutron net-delete my-private-network-2


#nova aggregate-remove-host my-aggregate-0 computenode0
#nova aggregate-delete my-aggregate-0

## Second Network Topology
On this second example, we will have the same network topology as the one constructed on our first example with the addition of a router connecting both private networks.



![Image](https://github.com/ml6973/Cloud-Tutorial/blob/master/OpenStack/figures/case2.png?raw=True)



In [ ]:
%%bash
. /home/stack/devstack/openrc admin admin
neutron net-create my-private-network-1
neutron net-create my-private-network-2
neutron subnet-create my-private-network-1 --name my-private-subnet-1 --gateway 192.180.0.1 192.180.0.0/24
neutron subnet-create my-private-network-2 --name my-private-subnet-2 --gateway 192.181.0.1 192.181.0.0/24

neutron router-create my-router-1

neutron router-interface-add my-router-1 my-private-subnet-1
neutron router-interface-add my-router-1 my-private-subnet-2

nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone nova --nic net-name=my-private-network-1 my-instance-1
nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone nova --nic net-name=my-private-network-2 my-instance-2


## Testing Network Connectivity

To perform network connectivity between instances, we will access the **my-router-1** and request a ping to be performed from **my_instance_1** to **my_instance_2**. To perform this action, run the following commands on the terminal:

In [ ]:
from IPython.display import IFrame
IFrame ("http://129.114.108.71:8888/terminals/1", width = 800, height=500)

## Output
In the given topology, each instance is  connected to a private network. In addition a router is connecting both of our private networks. As it was demonstrated, now we are able to observe the connectivity between instances in different networks.

## Second Network Topology Clean-up

The code shown below will remove the previously created topology.

In [ ]:
%%bash
. /home/stack/devstack/openrc admin admin

nova delete my-instance-1
nova delete my-instance-2

#Router 1, Public, Private 1,2
ROUTER_ID=$(neutron router-list | grep my-router-1 | grep -v '+' | grep -v 'external_gateway' | awk '{ print $2 }')
PRIVATE_SUBNET_ID=$(neutron subnet-list | grep '| my-private-subnet1' | awk '{print $2}')
neutron router-gateway-clear $ROUTER_ID

for i in `neutron port-list | grep -v '+' | grep -v 'fixed_ips' | awk '{print $2}'`;
do
  if [ "$i" != "|" ]
  then
    neutron port-update --device-owner None $i;
    neutron port-delete $i;
  fi
done;

neutron router-delete my-router-1
neutron net-delete my-private-network-1
neutron net-delete my-private-network-2


#nova aggregate-remove-host my-aggregate-0 computenode0
#nova aggregate-delete my-aggregate-0






## Creating our First Complex Network Topology
In our first complex topology, we will proceed to create one public network which we will connect to two routers. Each router will be connected to the two different private networks and at the end, we will create two instances and connect them to private networks that are attached to different routers.

![Image](https://github.com/cloudandbigdatalab/eLab-GUI-web-portal/blob/development/WebContent/resources/marketPlaceData/CloudNetworking/images/topology.png?raw=True)



In [ ]:
%%bash
. /home/stack/devstack/openrc admin admin

neutron net-create --shared --provider:network_type flat --provider:physical_network flat --router:external my-public-network

neutron subnet-create --name my-public-subnet-1 --gateway 10.10.10.1 --allocation-pool start=10.10.10.2,end=10.10.10.255 --dns-nameserver 8.8.8.8 my-public-network 10.10.0.0/16
neutron subnet-create --name my-public-subnet-2 --gateway 10.20.10.1 --allocation-pool start=10.20.10.2,end=10.20.10.255 --dns-nameserver 8.8.8.8 my-public-network 10.20.0.0/16

neutron net-create my-private-network-1
neutron net-create my-private-network-2
neutron net-create my-private-network-3
neutron net-create my-private-network-4

neutron subnet-create my-private-network-1 --name my-private-subnet-1 --gateway 192.168.0.1 192.168.0.0/24
neutron subnet-create my-private-network-2 --name my-private-subnet-2 --gateway 192.169.0.1 192.169.0.0/24
neutron subnet-create my-private-network-3 --name my-private-subnet-3 --gateway 192.170.0.1 192.170.0.0/24
neutron subnet-create my-private-network-4 --name my-private-subnet-4 --gateway 192.171.0.1 192.171.0.0/24

neutron router-create my-router-1
neutron router-create my-router-2

neutron router-interface-add my-router-1 my-public-subnet-1
neutron router-interface-add my-router-2 my-public-subnet-2

neutron router-interface-add my-router-1 my-private-subnet-1
neutron router-interface-add my-router-1 my-private-subnet-2
neutron router-interface-add my-router-2 my-private-subnet-3
neutron router-interface-add my-router-2 my-private-subnet-4

nova flavor-list
glance image-list

nova aggregate-create my-aggregate-0 compute-0
nova aggregate-add-host my-aggregate-0 computenode0

nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone nova --nic net-name=my-private-network-1 my-instance-01
nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone compute-0 --nic net-name=my-private-network-4 my-instance-02

nova list
nova availability-zone-list

## Output of The Code
As it can be observed, Neutron is a dynamic tool that can be used to build complex and scalable networks in our cloud environments within seconds.

## Third Network Topology Clean-up

The code shown below will remove the previously created topology.


In [ ]:
%%bash
. /home/stack/devstack/openrc admin admin

nova delete my-instance-01
nova delete my-instance-02

#Router 1, Public, Private 1,2
ROUTER_ID=$(neutron router-list | grep my-router-1 | grep -v '+' | grep -v 'external_gateway' | awk '{ print $2 }')
PRIVATE_SUBNET_ID=$(neutron subnet-list | grep '| my-private-subnet1' | awk '{print $2}')
PUBLIC_SUBNET_ID=$(neutron subnet-list | grep '| my-public-subnet' | awk '{print $2}')
neutron router-gateway-clear $ROUTER_ID

for i in `neutron port-list | grep -v '+' | grep -v 'fixed_ips' | awk '{print $2}'`;
do
  if [ "$i" != "|" ]
  then
    neutron port-update --device-owner None $i;
    neutron port-delete $i;
  fi
done;

neutron router-delete my-router-1
neutron net-delete my-public-network
neutron net-delete my-private-network-1
neutron net-delete my-private-network-2

#Router 2, Private 3,4
ROUTER_ID=$(neutron router-list | grep my-router-2 | grep -v '+' | grep -v 'external_gateway' | awk '{ print $2 }')
PRIVATE_SUBNET_ID=$(neutron subnet-list | grep '| my-private-subnet3' | awk '{print $2}')
neutron router-gateway-clear $ROUTER_ID

for i in `neutron port-list | grep -v '+' | grep -v 'fixed_ips' | awk '{print $2}'`;
do
  neutron port-update --device-owner None $i;
  neutron port-delete $i;
done;

neutron router-delete my-router-2
neutron net-delete my-private-network-3
neutron net-delete my-private-network-4

nova aggregate-remove-host my-aggregate-0 computenode0
nova aggregate-delete my-aggregate-0